In [ ]:
import random
import torch
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from sklearn.metrics import classification_report
from nltk.tokenize import word_tokenize
from gensim.models import Word2Vec
from utils.recorder_util import ModelResults

# enable tqdm in pandas
tqdm.pandas()

# set to True to use the gpu (if there is one available)
use_gpu = True



# select device
device = torch.device('cuda' if use_gpu and torch.cuda.is_available() else 'cpu')
print(f'device: {device.type}')

# random seed
seed = 1234

# set random seed
if seed is not None:
    print(f'random seed: {seed}')
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)


recorder = ModelResults("LSTM_W2V", "Bryan Bielawa", seed)


In [ ]:
train_df = pd.read_csv('../../../trainData/trainData.csv', header=None)
train_df

In [ ]:
#borrowing Noah's Word2Vec code
# Preprocessing text by:
#   1. lowercasing all letters and
#   2. splitting all strings into a list of words.

def preprocess_data(text):
    lower_text = text.lower()
    preprocessed = lower_text.split()
    return preprocessed

train_data["preprocessed_text"] = (train_data["text"]).apply(preprocess_data)
test_data["preprocessed_text"] = (test_data["text"]).apply(preprocess_data)

print("Preprocessed data:")
train_data.head()

In [ ]:
word2Vec = Word2Vec(
    sentences = train_data["preprocessed_text"], 
    vector_size = 100, 
    window = 5,
    min_count = 1,
    workers = 4)

def get_embedding(text, model):
    vectors = []
    for word in text:
        if word in model.wv:
            vectors.append(model.wv[word])
    if vectors:
        return np.mean(vectors, axis = 0)
    else:
        return np.zeros(100)